In [1]:
import re
import os
import itertools
import numpy as np
import pandas as pd
from typing import Any, Dict
import random
import yaml
import argparse
import glob
from tqdm import tqdm
from pathlib import Path

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores.utils import DistanceStrategy
import faiss
import pdfplumber
import torch
from sentence_transformers import CrossEncoder
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import warnings

2025-09-01 17:49:26.959264: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-01 17:49:26.969786: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756716566.981030 1677893 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756716566.984113 1677893 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-01 17:49:26.997538: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
base_path = "../data/law"
pdf_path = [base_path + f"/{i}" for i in os.listdir(base_path)]

pdf_text = {
    i : "" for i in os.listdir(base_path)
}

for i, key in zip(pdf_path, pdf_text.keys()):
    text = ""
    with pdfplumber.open(i) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n" 
    
    pdf_text[key] = text

Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss 

In [3]:
print(pdf_text.keys())

dict_keys(['부정경쟁방지 및 영업비밀보호에 관한 법률.pdf', '저작권법.pdf', '전자금융거래법.pdf', '특정 금융거래정보의 보고 및 이용 등에 관한 법률.pdf', '금융회사의 정보처리 업무 위탁에 관한 규정.pdf', '금융위원회의 설치 등에 관한 법률.pdf', '위치정보의 보호 및 이용 등에 관한 법률.pdf', '전자상거래 등에서의 소비자보호에 관한 법률.pdf', '정보통신망 이용촉진 및 정보보호 등에 관한 법률.pdf', '개인정보 보호법.pdf', '전자문서 및 전자거래 기본법.pdf', '산업기술의 유출방지 및 보호에 관한 법률.pdf', '소득세법.pdf', '여신전문금융업법.pdf', '통신비밀보호법.pdf', '전자서명법.pdf', '신용정보의 이용 및 보호에 관한 법률.pdf', '전자정부법.pdf', '클라우드컴퓨팅 발전 및 이용자 보호에 관한 법률.pdf', '정보보호산업의 진흥에 관한 법률 시행규칙.pdf', '가상자산 이용자 보호 등에 관한 법률.pdf', '전자금융감독규정.pdf', '전기통신사업법.pdf', '정보통신기반 보호법.pdf', '소프트웨어 진흥법 시행령.pdf', '사이버안보 업무규정.pdf', '소프트웨어 진흥법.pdf', '산업표준화법.pdf', '정보보호 관리등급 부여에 관한 고시.pdf', '은행법.pdf', '전자금융거래법 시행령.pdf', '금융기관검사및제재에관한규정.pdf', '정보보호산업의 진흥에 관한 법률.pdf'])


In [4]:
def split_by_article(text):
    # '제 N조' 앞에서 split
    pattern = r'(제\s*\d+조(?:의\d+)?\s*\([^)]+\)[\s\S]*?)(?=제\s*\d+조(?:의\d+)?\s*\([^)]+\)|\Z)'
    return re.findall(pattern, text)

pdf_split_data = []

for article, text in pdf_text.items():
    split_text = split_by_article(text)
    for i in split_text:
        pdf_split_data.append(f"{article[:-4]} {i}")

pdf_split_data[:5]

['부정경쟁방지 및 영업비밀보호에 관한 법률 제1조(목적) 이 법은 국내에 널리 알려진 타인의 상표ㆍ상호(商號) 등을 부정하게 사용하는 등의 부정경쟁행위와 타인\n의 영업비밀을 침해하는 행위를 방지하여 건전한 거래질서를 유지함을 목적으로 한다.\n[전문개정 2007. 12. 21.]\n',
 '부정경쟁방지 및 영업비밀보호에 관한 법률 제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2011. 12. 2., 2013. 7. 30., 2015. 1. 28., 2018. 4. 17.,\n2019. 1. 8., 2021. 12. 7., 2023. 3. 28., 2024. 2. 20.>\n1. “부정경쟁행위”란 다음 각 목의 어느 하나에 해당하는 행위를 말한다.\n가. 다음의 어느 하나에 해당하는 정당한 사유 없이 국내에 널리 인식된 타인의 성명, 상호, 상표, 상품의 용기ㆍ포\n장, 그 밖에 타인의 상품임을 표시한 표지(標識)(이하 이 목에서 “타인의 상품표지”라 한다)와 동일하거나 유사\n한 것을 사용하거나 이러한 것을 사용한 상품을 판매ㆍ반포(頒布) 또는 수입ㆍ수출하여 타인의 상품과 혼동하\n게 하는 행위\n1) 타인의 상품표지가 국내에 널리 인식되기 전부터 그 타인의 상품표지와 동일하거나 유사한 표지를 부정한\n목적 없이 계속 사용하는 경우\n2) 1)에 해당하는 자의 승계인으로서 부정한 목적 없이 계속 사용하는 경우\n나. 다음의 어느 하나에 해당하는 정당한 사유 없이 국내에 널리 인식된 타인의 성명, 상호, 표장(標章), 그 밖에 타\n인의 영업임을 표시하는 표지(상품 판매ㆍ서비스 제공방법 또는 간판ㆍ외관ㆍ실내장식 등 영업제공 장소의 전\n체적인 외관을 포함하며, 이하 이 목에서 “타인의 영업표지”라 한다)와 동일하거나 유사한 것을 사용하여 타인\n의 영업상의 시설 또는 활동과 혼동하게 하는 행위\n1) 타인의 영업표지가 국내에 널리 인식되기 전부터 그 타인의 영업표지와 동일하거나 유사한 표지를 부정한\n목적 없이 계속 사용하는 경우\n2

In [5]:
tech_data = pd.read_csv("../data/tech/tech_data.csv")

tech_array = list(tech_data['data'])
pdf_split_data.extend(tech_array)

In [6]:
ISMS_path = Path("../data/ISMS/merged_text")
ISMS_list = [ISMS_path / Path(f"{i}") for i in os.listdir(ISMS_path)]

ISMS_text = []
for i in ISMS_list:
    text = i.read_text(encoding="utf-8")
    ISMS_text.append(text)

pdf_split_data.extend(ISMS_text)

In [7]:
temp = []
for i in pdf_split_data:
    if len(i) > 100:
        temp.append(i)

pdf_split_data = temp

In [8]:
max_len = float("-inf")
min_len = float("inf")
total_len = 0

for i in pdf_split_data:
    length = len(i)

    if length > max_len:
        max_len = length

    if length < min_len:
        min_len = length

    total_len += length

print(f"total len: {total_len}")
print(f"max length : {max_len}")
print(f"min length : {min_len}")
print(f"mean length : {total_len/len(pdf_split_data)}")



total len: 136251560
max length : 7211
min length : 101
mean length : 2132.5626457560534


In [9]:
pdf_split_data_recursive = []

splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=500,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

for i in pdf_split_data:
    clean_text = re.sub(r'\s+', ' ', i).strip()
    clean_text = re.sub(r"[<\[].*?[>\]]", "", clean_text).strip()
    chunks = splitter.split_text(clean_text)
    pdf_split_data_recursive.extend(chunks)

pdf_split_data_recursive = [re.sub(r"[<\[].*?[>\]]", "", text).strip() for text in pdf_split_data_recursive]

pdf_split_data_recursive[:10]

['부정경쟁방지 및 영업비밀보호에 관한 법률 제1조(목적) 이 법은 국내에 널리 알려진 타인의 상표ㆍ상호(商號) 등을 부정하게 사용하는 등의 부정경쟁행위와 타인 의 영업비밀을 침해하는 행위를 방지하여 건전한 거래질서를 유지함을 목적으로 한다.',
 '부정경쟁방지 및 영업비밀보호에 관한 법률 제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다.  1. “부정경쟁행위”란 다음 각 목의 어느 하나에 해당하는 행위를 말한다. 가. 다음의 어느 하나에 해당하는 정당한 사유 없이 국내에 널리 인식된 타인의 성명, 상호, 상표, 상품의 용기ㆍ포 장, 그 밖에 타인의 상품임을 표시한 표지(標識)(이하 이 목에서 “타인의 상품표지”라 한다)와 동일하거나 유사 한 것을 사용하거나 이러한 것을 사용한 상품을 판매ㆍ반포(頒布) 또는 수입ㆍ수출하여 타인의 상품과 혼동하 게 하는 행위 1) 타인의 상품표지가 국내에 널리 인식되기 전부터 그 타인의 상품표지와 동일하거나 유사한 표지를 부정한 목적 없이 계속 사용하는 경우 2) 1)에 해당하는 자의 승계인으로서 부정한 목적 없이 계속 사용하는 경우 나. 다음의 어느 하나에 해당하는 정당한 사유 없이 국내에 널리 인식된 타인의 성명, 상호, 표장(標章), 그 밖에 타 인의 영업임을 표시하는 표지(상품 판매ㆍ서비스 제공방법 또는 간판ㆍ외관ㆍ실내장식 등 영업제공 장소의 전 체적인 외관을 포함하며, 이하 이 목에서 “타인의 영업표지”라 한다)와 동일하거나 유사한 것을 사용하여 타인 의 영업상의 시설 또는 활동과 혼동하게 하는 행위 1) 타인의 영업표지가 국내에 널리 인식되기 전부터 그 타인의 영업표지와 동일하거나 유사한 표지를 부정한 목적 없이 계속 사용하는 경우 2) 1)에 해당하는 자의 승계인으로서 부정한 목적 없이 계속 사용하는 경우 다. 가목 또는 나목의 혼동하게 하는 행위 외에 다음의 어느 하나에 해당하는 정당한 사유 없이 국내에 널리 인식 된 타인의 성명, 상호, 상표, 상품의 용기ㆍ포장, 그 밖에 타인의 상품 또는 영업임을 표

In [ ]:
embd_path = "../model/gte-multilingual-base"
embedding = HuggingFaceEmbeddings(
    model_name=embd_path,
    model_kwargs={"device": "cuda", "trust_remote_code": True},
    encode_kwargs={"normalize_embeddings": True},  # cosine에 필수 권장
)

docs = [Document(page_content=t, metadata={"doc_id": i}) for i, t in enumerate(pdf_split_data_recursive)]

vector_store = FAISS.from_documents(docs, embedding, distance_strategy=DistanceStrategy.COSINE)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [11]:
query = """전자금융거래법 제2조에 따르면, 전자금융업자가 제공하는 서비스에 해당하는 것은 무엇인가?
1 계좌 개설
2 대출 상담
3 신용카드 발급 신청
4 온라인 송금
5 보험 가입

[문서]
전자금융거래법 제2조(정의) 이 법에서 사용하는 용어의 정의는 다음과 같다.  1. “전자금융거래”라 함은 금융회사 또는 전자금융업자가 전자적 장치를 통하여 금융상품 및 서비스를 제공(이하 “전자금융업무”라 한다)하고, 이용자가 금융회사 또는 전자금융업자의 종사자와 직접 대면하거나 의사소통을 하 지 아니하고 자동화된 방식으로 이를 이용하는 거래를 말한다. 2. “전자지급거래”라 함은 자금을 주는 자(이하 “지급인”이라 한다)가 금융회사 또는 전자금융업자로 하여금 전자지 급수단을 이용하여 자금을 받는 자(이하 “수취인”이라 한다)에게 자금을 이동하게 하는 전자금융거래를 말한다. 3. “금융회사”란 다음 각 목의 어느 하나에 해당하는 기관이나 단체 또는 사업자를 말한다. 가. 「금융위원회의 설치 등에 관한 법률」 제38조제1호부터 제5호까지, 제7호 및 제8호에 해당하는 기관 나. 「여신전문금융업법」에 따른 여신전문금융회사 다. 「우체국예금ㆍ보험에 관한 법률」에 따른 체신관서 라. 「새마을금고법」에 따른 새마을금고 및 새마을금고중앙회 마. 그 밖에 법률의 규정에 따라 금융업 및 금융 관련 업무를 행하는 기관이나 단체 또는 사업자로서 대통령령이 정하는 자 4. “전자금융업자”라 함은 제28조의 규정에 따라 허가를 받거나 등록을 한 자(금융회사는 제외한다)를 말한다. 5. “전자금융보조업자”라 함은 금융회사 또는 전자금융업자를 위하여 전자금융거래를 보조하거나 그 일부를 대행하 는 업무를 행하는 자 또는 결제중계시스템의 운영자로서 「금융위원회의 설치 등에 관한 법률」제3조에 따른 금융 위원회(이하 “금융위원회”라 한다)가 정하는 자를 말한다. 6. “결제중계시스템”이라 함은 금융회사와 전자금융업자 사이에 전자금융거래정보를 전달하여 자금정산 및 결제에 관한 업무를 수행하는 금융정보처리운영체계를 말한다. 7. “이용자”라 함은 전자금융거래를 위하여 금융회사 또는 전자금융업자와 체결한 계약(이하 “전자금융거래계약”이 라 한다)에 따라 전자금융거래를 이용하는 자를 말한다. 8. “전자적 장치”라 함은 전자금융거래정보를 전자적 방법으로 전송하거나 처리하는데 이용되는 장치로서 현금자동 지급기, 자동입출금기, 지급용단말기, 컴퓨터, 전화기 그 밖에 전자적 방법으로 정보를 전송하거나 처리하는 장치 를 말한다. 9. “전자문서”라 함은 「전자문서 및 전자거래 기본법」 제2조제1호에 따른 작성, 송신ㆍ수신 또는 저장된 정보를 말 한다. 10. “접근매체”라 함은 전자금융거래에 있어서 거래지시를 하거나 이용자 및 거래내용의 진실성과 정확성을 확보하 기 위하여 사용되는 다음 각 목의 어느 하나에 해당하는 수단 또는 정보를 말한다. 가. 전자식 카드 및 이에 준하는 전자적 정보 나. 「전자서명법」 제2조제3호에 따른 전자서명생성정보 및 같은 조 제6호에 따른 인증서 다. 금융회사 또는 전자금융업자에 등록된 이용자번호 법제처 1 국가법령정보센터 전자금융거래법 라. 이용자의 생체정보 마. 가목 또는 나목의 수단이나 정보를 사용하는데 필요한 비밀번호 11. “전자지급수단”이라 함은 전자자금이체, 직불전자지급수단, 선불전자지급수단, 전자화폐, 신용카드, 전자채권 그 밖에 전자적 방법에 따른 지급수단을 말한다. 12. “전자자금이체”라 함은 지급인과 수취인 사이에 자금을 지급할 목적으로 금융회사 또는 전자금융업자에 개설된 계좌(금융회사에 연결된 계좌에 한한다. 이하 같다)에서 다른 계좌로 전자적 장치에 의하여 다음 각 목의 어느 하 나에 해당하는 방법으로 자금을 이체하는 것을 말한다. 가. 금융회사 또는 전자금융업자에 대한 지급인의 지급지시 나. 금융회사 또는 전자금융업자에 대한 수취인의 추심지시(이하 “추심이체”라 한다) 13. “직불전자지급수단”이라 함은 이용자와 가맹점간에 전자적 방법에 따라 금융회사의 계좌에서 자금을 이체하는 등의 방법으로 재화 또는 용역의 제공과 그 대가의 지급을 동시에 이행할 수 있도록 금융회사 또는 전자금융업자 가 발행한 증표(자금을 융통받을 수 있는 증표를 제외한다) 또는 그 증표에 관한 정보를 말한다.
"""

text = retriever.get_relevant_documents(query)

for i in text:
    print(i.page_content)

전자금융거래법 제2조(정의) 이 법에서 사용하는 용어의 정의는 다음과 같다.  1. “전자금융거래”라 함은 금융회사 또는 전자금융업자가 전자적 장치를 통하여 금융상품 및 서비스를 제공(이하 “전자금융업무”라 한다)하고, 이용자가 금융회사 또는 전자금융업자의 종사자와 직접 대면하거나 의사소통을 하 지 아니하고 자동화된 방식으로 이를 이용하는 거래를 말한다. 2. “전자지급거래”라 함은 자금을 주는 자(이하 “지급인”이라 한다)가 금융회사 또는 전자금융업자로 하여금 전자지 급수단을 이용하여 자금을 받는 자(이하 “수취인”이라 한다)에게 자금을 이동하게 하는 전자금융거래를 말한다. 3. “금융회사”란 다음 각 목의 어느 하나에 해당하는 기관이나 단체 또는 사업자를 말한다. 가. 「금융위원회의 설치 등에 관한 법률」 제38조제1호부터 제5호까지, 제7호 및 제8호에 해당하는 기관 나. 「여신전문금융업법」에 따른 여신전문금융회사 다. 「우체국예금ㆍ보험에 관한 법률」에 따른 체신관서 라. 「새마을금고법」에 따른 새마을금고 및 새마을금고중앙회 마. 그 밖에 법률의 규정에 따라 금융업 및 금융 관련 업무를 행하는 기관이나 단체 또는 사업자로서 대통령령이 정하는 자 4. “전자금융업자”라 함은 제28조의 규정에 따라 허가를 받거나 등록을 한 자(금융회사는 제외한다)를 말한다. 5. “전자금융보조업자”라 함은 금융회사 또는 전자금융업자를 위하여 전자금융거래를 보조하거나 그 일부를 대행하 는 업무를 행하는 자 또는 결제중계시스템의 운영자로서 「금융위원회의 설치 등에 관한 법률」제3조에 따른 금융 위원회(이하 “금융위원회”라 한다)가 정하는 자를 말한다. 6. “결제중계시스템”이라 함은 금융회사와 전자금융업자 사이에 전자금융거래정보를 전달하여 자금정산 및 결제에 관한 업무를 수행하는 금융정보처리운영체계를 말한다. 7. “이용자”라 함은 전자금융거래를 위하여 금융회사 또는 전자금융업자와 체결한 계약(이하 “전자금융거래계약”이 라 한다)에 따라 전자금융거래를 이용하는 자를 말한다. 8

/tmp/ipykernel_1677157/3900840525.py:12: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  text = retriever.get_relevant_documents(query)


In [12]:
device = "cuda"

# 예시: 쿼리 + 문서 리스트
docs = [i.page_content for i in text]

# Query-document 쌍 만들기
pairs = [(query, doc) for doc in docs]

def exp_normalize(x):
    return 1 / (1 + np.exp(-x))

model_path = "../model/bge-reranker-v2-m3"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.to(device)
model.eval()


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(8194, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, o

In [13]:
with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512).to(device)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    scores = exp_normalize(scores.cpu().numpy())
    
    for k, v in zip(pairs, scores):
        print(f"{k} : {v}")

('전자금융거래법 제2조에 따르면, 전자금융업자가 제공하는 서비스에 해당하는 것은 무엇인가?\n1 계좌 개설\n2 대출 상담\n3 신용카드 발급 신청\n4 온라인 송금\n5 보험 가입\n\n[문서]\n전자금융거래법 제2조(정의) 이 법에서 사용하는 용어의 정의는 다음과 같다.  1. “전자금융거래”라 함은 금융회사 또는 전자금융업자가 전자적 장치를 통하여 금융상품 및 서비스를 제공(이하 “전자금융업무”라 한다)하고, 이용자가 금융회사 또는 전자금융업자의 종사자와 직접 대면하거나 의사소통을 하 지 아니하고 자동화된 방식으로 이를 이용하는 거래를 말한다. 2. “전자지급거래”라 함은 자금을 주는 자(이하 “지급인”이라 한다)가 금융회사 또는 전자금융업자로 하여금 전자지 급수단을 이용하여 자금을 받는 자(이하 “수취인”이라 한다)에게 자금을 이동하게 하는 전자금융거래를 말한다. 3. “금융회사”란 다음 각 목의 어느 하나에 해당하는 기관이나 단체 또는 사업자를 말한다. 가. 「금융위원회의 설치 등에 관한 법률」 제38조제1호부터 제5호까지, 제7호 및 제8호에 해당하는 기관 나. 「여신전문금융업법」에 따른 여신전문금융회사 다. 「우체국예금ㆍ보험에 관한 법률」에 따른 체신관서 라. 「새마을금고법」에 따른 새마을금고 및 새마을금고중앙회 마. 그 밖에 법률의 규정에 따라 금융업 및 금융 관련 업무를 행하는 기관이나 단체 또는 사업자로서 대통령령이 정하는 자 4. “전자금융업자”라 함은 제28조의 규정에 따라 허가를 받거나 등록을 한 자(금융회사는 제외한다)를 말한다. 5. “전자금융보조업자”라 함은 금융회사 또는 전자금융업자를 위하여 전자금융거래를 보조하거나 그 일부를 대행하 는 업무를 행하는 자 또는 결제중계시스템의 운영자로서 「금융위원회의 설치 등에 관한 법률」제3조에 따른 금융 위원회(이하 “금융위원회”라 한다)가 정하는 자를 말한다. 6. “결제중계시스템”이라 함은 금융회사와 전자금융업자 사이에 전자금융거래정보를 전달하여 자금정산 및 결제에 관한 업무를 수행하는 

/home/aicontest/anaconda3/envs/construct/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [14]:
warnings.filterwarnings("ignore", category=FutureWarning)
# ----------------- 설정 -----------------
DOC_BATCH_SIZE = 10         # 문서 배치 크기 (환경에 맞게 조절)
MAX_LEN = 2048             # 토크나이저 max_length
SAVE_PATH = "../data/raginfo_all.csv"

# 모델을 평가 모드로
model.eval()

test_df = pd.read_csv("../data/test.csv")

submission_dict = {
    "ID": [],
    "Question": [],
    "RagInfo": [],
    "score": []
}

for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="RAG inference (batched per question)"):
    qid = row["ID"]
    question = row["Question"]

    # 1) 관련 문서 검색
    docs_obj = retriever.get_relevant_documents(question)
    docs = [d.page_content for d in docs_obj]

    # 관련 문서가 없을 때 방어
    if len(docs) == 0:
        submission_dict["ID"].append(qid)
        submission_dict["Question"].append(question)
        submission_dict["RagInfo"].append("")     # 빈 문자열 또는 None
        submission_dict["score"].append(0.0)
        continue

    # 2) 배치 단위로 로짓 수집
    all_logits = []
    for start in range(0, len(docs), DOC_BATCH_SIZE):
        end = start + DOC_BATCH_SIZE
        batch_docs = docs[start:end]
        batch_pairs = [(question, d) for d in batch_docs]

        with torch.no_grad():
            inputs = tokenizer(
                batch_pairs,
                padding=True,
                truncation=True,
                max_length=MAX_LEN,
                return_tensors="pt"
            ).to(device)

            # 모델 출력 (예: cross-encoder reranker 형태 가정)
            logits = model(**inputs, return_dict=True).logits.view(-1).float()
            all_logits.append(logits.detach().cpu().numpy())

        # (선택) VRAM 여유가 없다면 가끔 캐시 정리
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    # 3) 전체 로짓을 합치고, exp_normalize(softmax)로 스코어화
    all_logits = np.concatenate(all_logits, axis=0)   # shape: (num_docs,)
    scores = exp_normalize(all_logits)                # 사용자 정의 softmax 함수

    # 4) 최고 스코어 문서 선택
    best_idx = int(np.argmax(all_logits))
    best_doc = docs[best_idx]
    best_score = float(scores[best_idx])

    # 5) 결과 저장
    submission_dict["ID"].append(qid)
    submission_dict["Question"].append(question)
    submission_dict["RagInfo"].append(best_doc)
    submission_dict["score"].append(best_score)

# 6) CSV 저장
submission_df = pd.DataFrame(submission_dict)
submission_df.to_csv(SAVE_PATH, index=False, encoding="utf-8-sig")
print(f"Saved: {SAVE_PATH}")

RAG inference (batched per question): 100%|██████████| 515/515 [02:41<00:00,  3.19it/s]

Saved: ../data/raginfo_all.csv


In [15]:
test = pd.read_csv("../data/raginfo_all.csv")
test_list = [float(i) for i in test['score']]
print(sum(test_list)/len(test_list))
# 0.75

0.7532930450330859
